In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import hinge_loss, mean_absolute_error,mean_squared_log_error,median_absolute_error,mean_absolute_percentage_error,mean_poisson_deviance

In [3]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.4, random_state=44)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

1411


In [4]:

def get_metrics(y, p_y):
    metrics = {}
    try:
        metrics['mse'] = mse(y,p_y)
    except:
        pass
    try:
        metrics['hinge_loss'] = hinge_losss(y,p_y)
    except:
        pass
    try:
        metrics['mean_absolute_error'] = mean_absolute_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_squared_log_error'] = mean_squared_log_error(y,p_y)
    except:
        pass
    try:
        metrics['median_absolute_error'] = median_absolute_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_absolute_percentage_error'] = mean_absolute_percentage_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_poisson_deviance'] = mean_poisson_deviance(y,p_y)
    except:
        pass

    return metrics
    

# Handmade NN

In [5]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

# def build_model_convolutional(input_dim):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
#         tf.keras.layers.Dense(7*7*128, activation='relu'),
#         tf.keras.layers.Reshape((7, 7, 128)),
#         tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
#     ])
#     return model

In [6]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [7]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
# model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')
# model_conv.compile(optimizer='adam', loss='mse')

In [8]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=30)

Epoch 1/30
27/27 [==============================] - 2s 14ms/step - loss: 2788647.0000
Epoch 2/30
27/27 [==============================] - 0s 14ms/step - loss: 1986.8145
Epoch 3/30
27/27 [==============================] - 0s 14ms/step - loss: 1522.0913
Epoch 4/30
27/27 [==============================] - 0s 14ms/step - loss: 1520.4404
Epoch 5/30
27/27 [==============================] - 0s 14ms/step - loss: 1520.2556
Epoch 6/30
27/27 [==============================] - 0s 14ms/step - loss: 1520.1805
Epoch 7/30
27/27 [==============================] - 0s 14ms/step - loss: 1520.2094
Epoch 8/30
27/27 [==============================] - 0s 14ms/step - loss: 1520.2274
Epoch 9/30
27/27 [==============================] - 0s 15ms/step - loss: 1520.1019
Epoch 10/30
27/27 [==============================] - 0s 15ms/step - loss: 1520.1472
Epoch 11/30
27/27 [==============================] - 0s 15ms/step - loss: 1520.1292
Epoch 12/30
27/27 [==============================] - 0s 14ms/step - loss: 1520.092

In [9]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=30)

Epoch 1/30
27/27 [==============================] - 3s 14ms/step - loss: 1520.1564
Epoch 2/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.8827
Epoch 3/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.7241
Epoch 4/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.6101
Epoch 5/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.4581
Epoch 6/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.3538
Epoch 7/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.2706
Epoch 8/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.4215
Epoch 9/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.3490
Epoch 10/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.2090
Epoch 11/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.0745
Epoch 12/30
27/27 [==============================] - 0s 14ms/step - loss: 1519.0248
E

In [10]:
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
# Y_conv_predicted = model_conv.predict(np.array(X_test))
print(get_metrics(Y_predicted, Y_test_flat))
print(get_metrics(Y_ann_predicted, Y_test_flat))

18/18 [==============================] - 0s 2ms/step
{'mse': 74.29522716718571, 'mean_absolute_error': 0.12577708906821133, 'median_absolute_error': 0.07794308190445622, 'mean_absolute_percentage_error': 2.5399910750539783}
{'mse': 74.69558613056101, 'mean_absolute_error': 0.0751818996705495, 'median_absolute_error': 0.006599781854220603, 'mean_absolute_percentage_error': 12.85567560756224}


In [11]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
# Y_predicted

In [12]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
# Y_ann_predicted

# AutoKeras

In [13]:
import autokeras as ak
import numpy as np

In [14]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [15]:

input_node = ak.StructuredDataInput()
output_node = ak.DenseBlock()(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(inputs=input_node, outputs=output_node,overwrite=True, max_trials=100,metrics=['mae'],
)
reg.fit(train_set,epochs=11,validation_split=0.60)

# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)|

Trial 76 Complete [00h 00m 04s]
val_loss: 71.77154541015625

Best val_loss So Far: 71.20182800292969
Total elapsed time: 00h 04m 26s
INFO:tensorflow:Oracle triggered exit
Epoch 1/11
27/27 [==============================] - 2s 4ms/step - loss: 1521.1047 - mae: 0.7162
Epoch 2/11
27/27 [==============================] - 0s 4ms/step - loss: 1519.9017 - mae: 0.6162
Epoch 3/11
27/27 [==============================] - 0s 4ms/step - loss: 1518.5697 - mae: 0.5820
Epoch 4/11
27/27 [==============================] - 0s 4ms/step - loss: 1518.4628 - mae: 0.5557
Epoch 5/11
27/27 [==============================] - 0s 4ms/step - loss: 1516.8588 - mae: 0.5338
Epoch 6/11
27/27 [==============================] - 0s 4ms/step - loss: 1516.9115 - mae: 0.5222
Epoch 7/11
27/27 [==============================] - 0s 4ms/step - loss: 1515.5983 - mae: 0.5093
Epoch 8/11
27/27 [==============================] - 0s 4ms/step - loss: 1513.5770 - mae: 0.5008
Epoch 9/11
27/27 [==============================] - 0s 4ms/st

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [16]:
predicted_y = reg.predict(test_set)
print(reg.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))
predicted_y

18/18 [==============================] - 0s 2ms/step - loss: 74.3758 - mae: 0.2323
[74.37577819824219, 0.23233327269554138]
{'mse': 74.37577766666152, 'mean_absolute_error': 0.23233325722525575, 'median_absolute_error': 0.15384548096708697, 'mean_absolute_percentage_error': 85576247366.63478}


array([[-2.8058389e-01, -5.9937116e-02, -1.0602428e-03, ...,
        -2.8799770e-02,  5.1632982e-01,  3.7451500e+01],
       [ 9.0777636e-02, -9.6602201e-02, -5.4176174e-02, ...,
         3.7339166e-02,  8.2362592e-03,  1.1587290e+01],
       [ 5.6667488e-02, -6.5791957e-02,  3.0729316e-02, ...,
        -9.2120409e-02,  1.2409159e-01,  1.2201992e+01],
       ...,
       [ 1.5700724e-02, -1.5096664e-01, -3.2358751e-02, ...,
         1.5311030e-01, -1.1856837e-01,  1.5315228e+01],
       [-2.6006410e-01, -8.9214951e-02, -1.7049468e-01, ...,
         2.2291811e-02,  6.0457850e-01,  3.7931580e+01],
       [-2.4292919e-01, -8.8082880e-02, -1.6213116e-01, ...,
        -1.8241255e-02,  5.5519140e-01,  3.6256744e+01]], dtype=float32)

In [ ]:
regressor = ak.StructuredDataRegressor(max_trials=100, overwrite=True, metrics=['mse'],)
regressor.fit(train_set, epochs=11)
predicted_y = regressor.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(regressor.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))

Trial 4 Complete [00h 00m 08s]
val_loss: 214.6336669921875

Best val_loss So Far: 213.13284301757812
Total elapsed time: 00h 00m 35s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
True              |True              |structured_data_block_1/normalize
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
32                |32                |structured_data_block_1/dense_block_1/units_0
0                 |0                 |structured_data_block_1/dense_block_1/dropout
64                |64                |structured_data_block_1/dense_block_1/units_1
0                 |0                 |regression_head_1/dropout
adam              |adam              |optimizer
2e-05             |0.001             |learning_rate



In [ ]:
model = reg.export_model()
model.summary()

In [ ]:
model_regressor = regressor.export_model()
model_regressor.summary()

In [ ]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

In [ ]:
predicted_y = reg.predict(test_set)
print(reg.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))
predicted_y